# XGBoost with NSL-KDD

This notebook provides comparsion stats.  
The NSL-KDD version used is the [preprocessed one by the University of New Brunswick, Canada](http://www.unb.ca/cic/datasets/nsl.html).

In [1]:
import os
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

## Data loading and prep

As we've pickled the normalized and encoded dataset, we only need to load these pickles to get the Pandas DataFrames back.  
**Hint**: If you miss the pickles, go ahead and run the notebook named *Pickle-NSL-KDD-ipynb*

In [2]:
def load_df(filename):
    filepath = os.path.join('NSL_KDD', filename+'.pkl')
    return pd.read_pickle(filepath)

In [3]:
kdd_train_data = load_df('kdd_train_data')
kdd_test_data = load_df('kdd_test_data')
kdd_train_labels = load_df('kdd_train_labels')
kdd_test_labels = load_df('kdd_test_labels')

In [4]:
kdd_train_data.tail()

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,protocol_type,service,flag
125968,0.000000,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.360078,0.048924,1.0,1.0,0.0,0.0,0.14,0.06,0.0,1.0,0.098039,0.10,0.06,0.00,0.0,1.00,1.0,0.00,0.0,0.0,0.028986,0.1
125969,0.000186,7.608895e-08,1.106923e-07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.003914,0.003914,0.0,0.0,0.0,0.0,1.00,0.00,0.0,1.0,0.956863,0.96,0.01,0.01,0.0,0.00,0.0,0.00,0.0,0.5,0.028986,0.0
125970,0.000000,1.616709e-06,2.931438e-07,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001957,0.001957,0.0,0.0,0.0,0.0,1.00,0.00,0.0,1.0,0.117647,0.12,0.06,0.00,0.0,0.72,0.0,0.01,0.0,0.0,0.217391,0.0
125971,0.000000,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.281800,0.015656,1.0,1.0,0.0,0.0,0.06,0.05,0.0,1.0,0.031373,0.03,0.05,0.00,0.0,1.00,1.0,0.00,0.0,0.0,0.507246,0.1
125972,0.000000,1.094232e-07,0.000000e+00,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001957,0.001957,0.0,0.0,0.0,0.0,1.00,0.00,0.0,1.0,0.301961,0.30,0.03,0.30,0.0,0.00,0.0,0.00,0.0,0.0,0.000000,0.0


The paper mentions that they only use six features of the full dataset which is why we filter the dataframes for these.

In [5]:
# only keep columns that are actually used
used_fields = ['duration', 'protocol_type', 'src_bytes', 'dst_bytes', 'count', 'srv_count']
kdd_train_data = kdd_train_data.filter(used_fields)
kdd_test_data = kdd_test_data.filter(used_fields)
kdd_train_data.tail()

,duration,protocol_type,src_bytes,dst_bytes,count,srv_count
125968,0.000000,0.0,0.000000e+00,0.000000e+00,0.360078,0.048924
125969,0.000186,0.5,7.608895e-08,1.106923e-07,0.003914,0.003914
125970,0.000000,0.0,1.616709e-06,2.931438e-07,0.001957,0.001957
125971,0.000000,0.0,0.000000e+00,0.000000e+00,0.281800,0.015656
125972,0.000000,0.0,1.094232e-07,0.000000e+00,0.001957,0.001957


### Test Set

Encode the lables to a numeric list

In [6]:
kdd_train_labels.head()

,label,difficulty_level,label_encoded
0,normal,20,1
1,normal,15,1
2,neptune,19,2
3,normal,21,1
4,normal,21,1


In [7]:
def f(x):
    return 0 if x ==1 else 1 # FIXME: I should really load the tokenizer alphabet and check which int is benign
f = np.vectorize(f)

# We only want to know if it's benign or not, so we switch to 0 or 1
kdd_train_labels = f(kdd_train_labels['label_encoded'].values)
kdd_test_labels = f(kdd_test_labels['label_encoded'].values)

In [8]:
kdd_train_labels[:5]

array([0, 0, 1, 0, 0])

## Runtime Preqs

These are the settings Tang et al. defined in their paper.

In [9]:
batch_size = 10
epochs = 100
learn_rate = 0.001

In [10]:
from datetime import datetime
from os.path import exists, join

print('Using set globals: ')
print('Batch Size:\t', batch_size)
print('Epochs:\t\t', epochs)
print('Learn rate:\t', learn_rate)

Using set globals: 
Batch Size:	 10
Epochs:		 100
Learn rate:	 0.001


In [11]:
print("No of train entries:\t", len(kdd_train_data))
print("No of train labels:\t", len(kdd_train_labels))
print("-----------")
print("No of test entries:\t", len(kdd_test_data))
print("No of test labels:\t", len(kdd_test_labels))

No of train entries:	 125973
No of train labels:	 125973
-----------
No of test entries:	 22543
No of test labels:	 22543


## Building and Training the Model

In [12]:
from xgboost import XGBClassifier

xgb = XGBClassifier(
    n_estimators=100,
    n_jobs=-1,
    random_state=0
)

xgb.fit(kdd_train_data, kdd_train_labels)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=-1, nthread=None, objective='binary:logistic',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [13]:
from sklearn.metrics import accuracy_score

kdd_test_predicitions = xgb.predict(kdd_test_data)
predicted = [round(value) for value in kdd_test_predicitions]

accuracy = accuracy_score(kdd_test_labels, predicted)

/home/matzilla/Entwicklung/Master_IPython/venv/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [14]:
print('Mean accuracy score: {}'.format(accuracy))

Mean accuracy score: 0.7669786629996007
